# Decoradores

¿Quién podría explicar lo que está pasando aquí?:

### Funciones (first-class object)

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(funcion):
    return funcion("Bob")

In [2]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

In [8]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [11]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

In [6]:
first = parent(1)
second = parent(2)


# descomentar las siguientes líneas y mirad que va pasando
# first
# second

# first()
# second()

### Un decorador básico

In [20]:
def my_decorator_printantesdespues(func):
    def wrapper():
        print("Ocurre algo ANTES de llamar a la función.")
        print(f"el valor de retorno es: {func()}")
        print("Ocurre algo DESPUÉS de llamar a la función.")

    return wrapper

In [20]:
def say_whee():
    print("Whee!")


say_whee_decorada = my_decorator_printantesdespues(say_whee)


@my_decorator_printantesdespues
def say_whee():
    print("Whee!")

In [22]:
say_whee_decorada()

Ocurre algo ANTES de llamar a la función.
Whee!
el valor de retorno es: None
Ocurre algo DESPUÉS de llamar a la función.


In [23]:
@my_decorator_printantesdespues
def funcion():
    
    return "cadena"

In [24]:
funcion()

Ocurre algo ANTES de llamar a la función.
el valor de retorno es: cadena
Ocurre algo DESPUÉS de llamar a la función.


In [11]:
@my_decorator_printantesdespues
def decir_hola():
    print("Holaaa")

In [12]:
decir_hola()

Ocurre algo ANTES de llamar a la función.
Holaaa
el valor de retorno es: None
Ocurre algo DESPUÉS de llamar a la función.


In [25]:
permitido = True


def pedir_permiso(func):
    def wrapper():
        if permitido:
            print("Adelante!")
            func()
        else:
            print("Pues va a ser que no...")
            pass  # No se puede :(

    return wrapper


@pedir_permiso
def hacer_algo():
    print("Voy a bañarme!")

In [28]:
permitido = False


hacer_algo()

Pues va a ser que no...


In [36]:
# decorado que permite argumentos
def pedir_permiso_arg(permitido=True):
    def decorador_real(func):
        def wrapper(*args, **kwargs):
            if permitido:
                print("Adelante!")
                func(*args, **kwargs)
            else:
                print("Pues va a ser que no...")
                pass  # No se puede :(

        return wrapper
    
    return decorador_real


@pedir_permiso_arg
def hacer_algo():
    print("Voy a bañarme!")

In [37]:
hacer_algo()

Adelante!
Voy a bañarme!


Como hacer correctamente un decorador que acepte argumentos pero que funcione también si no le pasamos ningun argumento. Es decir, los argumentos son opcionales.

In [ ]:
# source: https://florimond.dev/blog/articles/2019/09/python-optionally-parametrized-decorators/
# also Python Cookbook: Recipes for Mastering Python 3
# Book by Brian K. Jones and David M. Beazley
# https://stackoverflow.com/questions/48098569/use-of-functools-partial-in-a-decorator-that-attaches-function-as-attribute-of-o

import functools


def logged(func=None, decimals: int = None):
    if func is None:
        print(f"Called with decimals={decimals}")
        return functools.partial(logged, decimals=decimals)

    print(f"Called without parameters, func={func}.")

    @functools.wraps(func)
    def decorated(*args, **kwargs):
        print(f"{func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        logged_result = result if decimals is None else round(result, decimals)
        print(f"Result: {logged_result}")
        return result

    return decorated

In [42]:
from datetime import datetime
import functools

def not_during_the_night(func):
    @functools.wraps(func)
    def wrapper():
        if 20 <= datetime.now().hour < 11:
            func()
        else:
            print("es de noche")
            pass
    return wrapper


def say_whee():
    """Dice whee."""
    
    print("Whee!")

# no permite ejecutar la función a ciertas horas
say_whee = not_during_the_night(say_whee)

In [39]:
say_whee()

es de noche


In [20]:
say_whee()

es de noche


`*args` y `**kwargs`

In [46]:
def f(*args, **kwargs):
    print(args)
    print(kwargs)

In [48]:
f(1,2,3,4, ultimo=5)

(1, 2, 3, 4)
{'ultimo': 5}


In [1]:
import time

In [36]:
import time
import functools


def timer(func):
    
    num_llamadas = 0
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        valor = func(*args, **kwargs)
        end_time = time.perf_counter()
        total = end_time - start_time
        
        nonlocal num_llamadas
        num_llamadas += 1
        
        print("Llamdas:", num_llamadas)
        print(f"La funcion {func.__name__} se ha ejecutado en:", f"{total:.2f}", f"usando los parámentros {args, kwargs}")
        
        return valor
    
    return wrapper

In [37]:
@timer
def suma(a, b):
    
    return a + b

In [38]:
suma(2, 4)

Llamdas: 1
La funcion suma se ha ejecutado en: 0.00 usando los parámentros ((2, 4), {'rm': 2})


6

## Ejercicio

Crear un decorador llamado `@timer`.

El decorados me dice el tiempo que ha tardado en ejecutarse está funcion (usando `print()`).

### ¿Qué es eso de @functools.wrap?

`@functools.wraps` es un decorador que preserva la información de la función original (metadatos: docstring, etc)

### Usando clases para crear decoradores

La mayoría de las veces no será necesario 🙅🏻‍♀️

¿Cómo haríais un decorador que almacena el número de veces que la función ha sido llamada?

In [21]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def haz_algo():
    print("Whee!")

In [22]:
import time

In [23]:
haz_algo()

Call 1 of 'haz_algo'
Whee!


El método `.__ init __ ()` debe almacenar una referencia a la función y puede hacer cualquier otra inicialización necesaria. Se llamará al método `.__ call __ ()` en lugar de la función decorada. Hace esencialmente lo mismo que la función `wrapper()` en nuestros ejemplos anteriores. Tenga en cuenta que debe usar la función `functools.update_wrapper()` en lugar de `@functools.wraps`.

In [24]:
# no look before


# source: https://python-3-patterns-idioms-test.readthedocs.io/en/latest/PythonDecorators.html#what-can-you-do-with-decorators
# PythonDecorators/decorator_with_arguments.py

class decorador_con_argumentos(object):
    def __init__(self, arg1, arg2, arg3):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print("Inside __init__()")
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3

    def __call__(self, f):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print("Inside __call__()")

        def wrapped_f(*args):
            print("Inside wrapped_f()")
            print("Decorator arguments:", self.arg1, self.arg2, self.arg3)
            f(*args)
            print("After f(*args)")

        return wrapped_f


@decorador_con_argumentos("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print("sayHello arguments:", a1, a2, a3, a4)


print("After decoration")

print("Preparing to call sayHello()")
sayHello("say", "hello", "argument", "list")
print("after first sayHello() call")
sayHello("a", "different", "set of", "arguments")
print("after second sayHello() call")

Inside __init__()
Inside __call__()
After decoration
Preparing to call sayHello()
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: say hello argument list
After f(*args)
after first sayHello() call
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: a different set of arguments
After f(*args)
after second sayHello() call


### Ejercicios 🎉

`@do_twice`: ejecutar la función 2 veces  
`@timer`: contar cuanto tiempo tarda en ejecutarse la función  
> `@logger`: guardar nombre de la función y hora en una lista  

`@slowdown`: parar un tiempo X entre cada ejecución  
`@cache`: guardar el valor de la función para esos argumentos  
* ¿Alguien puede conseguir lo mismo con solo 1 línea de código?

`@repeat(n=4)`: igual que `@do_twice` pero ahora podemos elegir el número de veces que queremos que se repita

*Extra:*  
`@debug`: hacer print de los argumentos con los que se ha llamado la función y el nombre de la función cada vez que se llama

In [47]:
# do twice
import functools


def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)

    return wrapper_do_twice

In [57]:
def repeat(n=2):
    def decorador_real(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                func(*args, **kwargs)

        return wrapper
    
    return decorador_real

In [ ]:
def printar(a, n=None):
    if n:
        for _ in range(n):
            do()

In [58]:
@repeat(n=4)
def printar(a):
    print(a)

In [59]:
printar("repitiendo")

repitiendo
repitiendo
repitiendo
repitiendo


In [80]:
def optional_repeat(func):
    def wrapper(*args, **kwargs):
        
        n = kwargs.get("repeticiones", 2)
        
        if n:
            for _ in range(n):
                func(*args, **kwargs)
        else:
            func(*args, **kwargs)

    return wrapper

In [86]:
@optional_repeat
def printar(a, repeticiones=None):
    print(a)

In [88]:
printar("done")

done
done


In [51]:
# slow down
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

In [47]:
import time

def memoize(function):
    
    memo = {}

    def wrapper(*args):
        if args in memo:
            return memo[args] # = valor retornado
        else:
            rv = function(*args)
            memo[args] = rv
            return rv

    return wrapper


@memoize
def fibonacci(n):
    time.sleep(2)
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


def fibonacci_nocache(n):
    time.sleep(2)
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
%time

fibonacci(20)

In [ ]:
%time

fibonacci_nocache(190)

In [52]:
@slow_down
def funcion_lenta():
    "ejectuar despacio"
    
    print("ejecutada")
    pass

In [53]:
def funcion_normal():
    "ejecutar rapido"
    
    print("ejecutada")
    pass

In [54]:
funcion_normal()

ejecutada


In [55]:
funcion_lenta()

ejecutada


In [ ]:
# timer
import functools


def timer(func):
    """Print the runtime of the decorated function"""

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

In [ ]:
import time

In [3]:
@timer
def function():
    "Documentación de la función"
    
    print("no soy mut util")

In [10]:
function()

no soy mut util
Finished 'function' in 0.0001 secs


In [ ]:
# cache
import functools


def cache(func):
    """Keep a cache of previous function calls"""

    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]

    wrapper_cache.cache = dict()
    return wrapper_cache

In [ ]:
# debug
import functools


def debug(func):
    """Print the function signature and return value"""

    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]  # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)  # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")  # 4
        return value

    return wrapper_debug

In [56]:
from functools import lru_cache

Más información y fuentes:

* https://realpython.com/primer-on-python-decorators 👈🏼
* https://dbader.org/blog/python-decorators
* https://blog.miguelgrinberg.com/post/the-ultimate-guide-to-python-decorators-part-i-function-registration
* https://stackoverflow.com/questions/10294014/python-decorator-best-practice-using-a-class-vs-a-function 📖
* https://www.youtube.com/watch?v=MjHpMCIvwsY 📹

Ricardo Ander-Egg Aguilar

* 🖥: https://ricardoanderegg.com/
* 🐦: https://twitter.com/ricardoanderegg
* 👨🏻‍🎓: https://www.linkedin.com/in/ricardoanderegg/